<a href="https://colab.research.google.com/github/ultra151/cv/blob/main/18_%EA%B0%9D%EC%B2%B4%ED%83%90%EC%A7%80YOLO%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install torch torchvision

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 12.9 MB/s eta 0:00:00


In [ ]:
%cd /content/gdrive/MyDrive//pytest_img/YOLO/

/content/gdrive/MyDrive/pytest_img/YOLO


In [ ]:
!ls

data.yaml    nets		  README.roboflow.gdoc	test   valid	    yolov5
gangnam.jpg  README.dataset.gdoc  sample.mp4		train  walking.mp4  yolov5s.pt


In [ ]:
from ultralytics import YOLO
model = YOLO("../models/yolov8m.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 49.7M/49.7M [00:00<00:00, 141MB/s]


In [ ]:
seconds = 1

video_path = '/content/gdrive/MyDrive/pytest_img/YOLO/sample.mp4'
output_dir = '/content/gdrive/MyDrive/pytest_img/YOLO/results'

In [ ]:
import torch

def predict(frame, iou=0.7, conf=0.25):
    results = model(
        source = frame,
        device = "0" if torch.cuda.is_available() else "cpu",
        iou = iou,
        conf = conf,
        verbose = False,
    )
    return results[0]

In [ ]:
import cv2
import numpy as np

def draw_boxes(results, frame):
    for box in results.boxes:
        x1, y1, x2, y2, score, classes = box.data.squeeze().numpy()
        label = model.names[int(classes)]
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 3)
        cv2.putText(frame, f'{label} {score:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 2)
    return frame

In [ ]:
import cv2

capture = cv2.VideoCapture(video_path)

if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
    capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

fps = capture.get(cv2.CAP_PROP_FPS)
total_frames = int(fps * seconds)
print('total 프레임:', total_frames)

total 프레임: 25


In [ ]:
import os

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

frame_count = 0
data = []
while frame_count < total_frames:
    ok, frame = capture.read()
    if not ok:
        print("프레임 읽기에 실패했습니다. 종료.")
        break
    result = predict(frame)
    frame = draw_boxes(result, frame)
    output_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(output_path, frame)
    print("Saved to:", output_path)

    for box in result.boxes:
        x1, y1, x2, y2, score, cls_id = box.data.squeeze().numpy()
        data.append([frame_count, x1, y1, x2, y2, np.round(score, 3), int(cls_id)])

    frame_count += 1
    #key = cv2.waitKey(10)
    #if key == ord('q'):
    #    print("사용자가 종료를 요청했습니다.")
    #    break

capture.release()
#cv2.destroyAllWindows()

Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0000.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0001.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0002.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0003.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0004.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0005.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0006.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0007.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0008.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0009.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0010.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0011.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/results/frame_0012.jpg
Saved to: /content/gdrive/MyDrive/pytest_img/YOLO/r

In [ ]:
import pandas as pd
df = pd.DataFrame(data, columns=['frame', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class_id'])
df

,frame,xmin,ymin,xmax,ymax,confidence,class_id
0,0,761.961487,63.420273,1317.433228,571.924988,0.931,0
1,0,365.622437,484.062744,1672.197144,817.899902,0.860,8
2,1,763.614990,63.289032,1319.324829,574.400269,0.928,0
3,1,361.093811,488.134918,1598.921265,819.122925,0.891,8
4,2,769.686768,62.607101,1321.657104,578.593628,0.930,0
5,2,363.985474,488.892517,1606.399048,818.776062,0.881,8
6,2,885.452271,218.280670,1185.341797,495.739655,0.458,24
7,3,772.311035,61.516983,1322.679932,582.230591,0.931,0
8,3,365.891418,494.700165,1588.260986,816.782288,0.866,8
9,3,885.941711,217.941299,1189.287720,500.989014,0.540,24
